In [30]:
import requests
import json
import pandas as pd
import plotly.graph_objs as go
%matplotlib inline

# Extraction

In [14]:
dataset = "abscs"
params="NAME,PAYANN,EMP"
location="state:*"
key="b2c289dbb192d92ff73949ae8d4157f0731334e5"
datasets = []
state_abbrv = pd.read_csv("Resources/state_abbrv.csv")

for year in range(2017, 2020):
    url=f"https://api.census.gov/data/{year}/{dataset}?get={params}&for={location}&key={key}"
    data = json.loads(requests.get(url).text)
    headers = data.pop(0)
    data = pd.DataFrame(data, columns=headers)
    datasets.append(data)



# Transformation

In [25]:
dataset_transformed = []
year = 2017

for df in datasets:
    data_df = df.merge(state_abbrv, how='inner', right_on='State', left_on="NAME")
    data_df.drop(columns=['NAME', 'state'], inplace=True)
    data_df[f'Annual Pay Per Employee ({year})'] = pd.to_numeric(data_df['PAYANN']) / pd.to_numeric(data_df['EMP']) * 1000
    data_df.drop(columns=['PAYANN', 'EMP', 'State'], inplace=True)
    dataset_transformed.append(data_df)
    year += 1


In [26]:
merged_df = dataset_transformed[0].merge(dataset_transformed[1], how='inner', on = 'State_abbr')
merged_df = merged_df.merge(dataset_transformed[2], how='inner', on = 'State_abbr')
merged_df['Avg (2017-2019)'] = merged_df[['Annual Pay Per Employee (2017)', 'Annual Pay Per Employee (2018)', 'Annual Pay Per Employee (2019)']].mean(axis=1)
merged_df['Change 2017-2019'] = (pd.to_numeric(merged_df['Annual Pay Per Employee (2019)']) - pd.to_numeric(merged_df['Annual Pay Per Employee (2017)'])) / (pd.to_numeric(merged_df['Annual Pay Per Employee (2017)'])) * 100


# Plots

### US Average Annual Pay Per Employee by State (2017-2019)

In [40]:
fig = go.Figure(data=go.Choropleth(
    locations=merged_df['State_abbr'], # Spatial coordinates
    z = merged_df['Avg (2017-2019)'].astype(int), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'mint',
    colorbar_title = "USD",
))

fig.update_layout(
    title_text = 'US Average Annual Pay Per Employee by State (2017-2019) ',
    geo_scope='usa', # limit map scope to USA
  )

### Percentage Change in Average Annual Pay Per Employee by State (2017-2019)

In [39]:
fig = go.Figure(data=go.Choropleth(
    locations=merged_df['State_abbr'], # Spatial coordinates
    z = merged_df['Change 2017-2019'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = 'purples',
    colorbar_title = "USD",
))

fig.update_layout(
    title_text = 'Percentage Change in Average Annual Pay Per Employee by State (2017-2019)',
    geo_scope='usa', # limit map scope to USA
  )